가장 성능이 높게 나온 LGBM코드
로그 취해서 스케일링 하고 모델 하이퍼파라미터도 내용 더 추가해서 튜닝 함

In [2]:
import pandas as pd
import numpy as np
import datetime
from scipy.stats import pearsonr
from statsmodels.tsa.stattools import grangercausalitytests
from lightgbm import LGBMRegressor
from tqdm import tqdm

# --- [S1] 공행성 쌍 탐색 함수 정의 (변경 없음) ---

def safe_corr_with_pvalue(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0, 1.0
    corr, p_value = pearsonr(x, y)
    return float(corr), float(p_value) # 피어슨 상관계수, p-value

def granger_test(x, y, max_lag=6):
    """
    x가 y를 Granger-cause 하는지 검증
    """
    try:
        data = pd.DataFrame({'x': x, 'y': y})
        test_result = grangercausalitytests(data[['y', 'x']],
                                            maxlag=max_lag,
                                            verbose=False)
        # 각 lag의 p-value 중 최소값 반환
        p_values = [test_result[lag][0]['ssr_ftest'][1]
                      for lag in range(1, max_lag+1)]
        return min(p_values)
    except:
        return 1.0

def calculate_score_advanced(corr, p_value, granger_p, lag):
    """
    정교한 공행성 점수 계산
    """

    # 1. 상관계수 점수 (0~50점) - 비선형 스케일링
    if abs(corr) >= 0.8:
        corr_score = 50
    elif abs(corr) >= 0.7:
        corr_score = 45
    elif abs(corr) >= 0.6:
        corr_score = 38
    elif abs(corr) >= 0.5:
        corr_score = 30
    else:
        corr_score = abs(corr) * 50  # 0.5 미만은 선형

    # 2. 통계적 유의성 점수 (0~25점) - 더 세분화
    if p_value < 0.001:
        sig_score = 25  # 매우 강한 유의성
    elif p_value < 0.01:
        sig_score = 20
    elif p_value < 0.05:
        sig_score = 12
    else:
        sig_score = 5  # 약한 유의성도 약간 점수

    # 3. Granger 인과성 점수 (0~20점) - 더 세분화
    if granger_p < 0.001:
        granger_score = 20  # 매우 강한 인과성
    elif granger_p < 0.01:
        granger_score = 16
    elif granger_p < 0.05:
        granger_score = 10
    elif granger_p < 0.10:
        granger_score = 5  # 약한 인과성
    else:
        granger_score = 0

    # 4. Lag 점수 (0~5점) - 비선형, 너무 짧거나 긴 lag 패널티
    if lag == 2 or lag == 3:
        lag_score = 5  # 최적
    elif lag == 1 or lag == 4:
        lag_score = 4  # 양호
    elif lag == 5:
        lag_score = 2  # 보통
    else:  # lag >= 6
        lag_score = 0  # 너무 김

    total_score = corr_score + sig_score + granger_score + lag_score

    return total_score

def find_comovement_pairs(
    pivot,
    max_lag=6,
    min_nonzero=12,
    corr_threshold=0.4,
    score_threshold=40
):
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    n_months = len(months)
    results = []

    for i, leader in tqdm(enumerate(items), total=len(items)):
        x = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(x) < min_nonzero:
            continue

        candidates = []

        for follower in items:
            if follower == leader:
                continue

            y = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None
            best_corr = 0.0
            best_p_value = 1.0

            # lag 탐색
            for lag in range(1, max_lag + 1):
                if n_months <= lag:
                    continue

                corr, p_value = safe_corr_with_pvalue(x[:-lag], y[lag:])

                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag
                    best_p_value = p_value

            # 기본 임계값 통과 시
            if best_lag is not None and abs(best_corr) >= corr_threshold:
                # Granger test
                granger_p = granger_test(x, y, max_lag=best_lag)

                # 종합 점수 계산
                score = calculate_score_advanced(
                    best_corr, best_p_value, granger_p, best_lag
                )

                # 점수 임계값 통과 시 후보에 추가
                if score >= score_threshold:
                    candidates.append({
                        "following_item_id": follower,
                        "best_lag": best_lag,
                        "max_corr": best_corr,
                        "p_value": best_p_value,
                        "granger_p_value": granger_p,
                        "comovement_score": score,
                    })

        # 알파벳 순으로 정렬
        candidates.sort(key=lambda x: x['following_item_id'])

        for candidate in candidates:
            results.append({
                "leading_item_id": leader,
                "following_item_id": candidate["following_item_id"],
                "best_lag": candidate["best_lag"],
                "max_corr": candidate["max_corr"]
            })

    pairs = pd.DataFrame(results)
    return pairs


# --- (2. 학습 데이터 구축 - [수정됨] Sin/Cos 변환) ---
def build_training_data(pivot, pairs, months_dt):
    months = months_dt
    n_months = len(months)
    rows = []

    for row in pairs.itertuples(index=False):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)
        if leader not in pivot.index or follower not in pivot.index:
            continue
        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        for t in range(max(lag, 12), n_months - 1):
            b_t = b_series[t]
            b_t_1 = b_series[t - 1]
            a_t_lag = a_series[t - lag]
            b_t_plus_1 = b_series[t + 1]

            b_t_12 = b_series[t - 12]
            b_roll_mean_3 = (b_series[t] + b_series[t-1] + b_series[t-2]) / 3

            # [수정 1] Sin/Cos 변환 적용
            target_month = months[t + 1].month
            month_sin = np.sin(2 * np.pi * target_month / 12)
            month_cos = np.cos(2 * np.pi * target_month / 12)


            rows.append({
                "b_t": b_t,
                "b_t_1": b_t_1,
                "a_t_lag": a_t_lag,
                "max_corr": corr,
                "best_lag": float(lag),
                "b_t_12": b_t_12,
                "b_roll_mean_3": b_roll_mean_3,
                # "month": float(target_month), # <-- 기존 피처
                "month_sin": month_sin,       # <-- [수정] 새 피처
                "month_cos": month_cos,       # <-- [수정] 새 피처
                "target": b_t_plus_1,
            })
    df_train = pd.DataFrame(rows)
    return df_train


# --- (4. 예측 - [수정됨] Sin/Cos 변환 및 Log 역변환) ---
def predict(pivot, pairs, reg, months_dt):
    months = months_dt
    n_months = len(months)
    t_last = n_months - 1
    t_prev = n_months - 2
    preds = []

    # [수정 2] 예측할 달(8월)의 Sin/Cos 값 미리 계산
    target_month = months[-1].month + 1 if months[-1].month < 12 else 1
    target_month_sin = np.sin(2 * np.pi * target_month / 12)
    target_month_cos = np.cos(2 * np.pi * target_month / 12)

    for row in tqdm(pairs.itertuples(index=False)):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)
        if leader not in pivot.index or follower not in pivot.index:
            continue
        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        if t_last - lag < 0 or t_last < 12:
            continue

        b_t = b_series[t_last]
        b_t_1 = b_series[t_prev]
        a_t_lag = a_series[t_last - lag]
        b_t_12 = b_series[t_last - 12]
        b_roll_mean_3 = (b_series[t_last] + b_series[t_last-1] + b_series[t_last-2]) / 3

        # [수정 3] X_test 구성 시 Sin/Cos 피처 사용
        X_test = np.array([[
            b_t, b_t_1, a_t_lag, corr, float(lag),
            b_t_12, b_roll_mean_3,
            target_month_sin, target_month_cos # <-- [수정]
        ]])

        # 모델이 log 스케일의 값을 예측
        y_pred_log = reg.predict(X_test)[0]

        # Log 역변환 (expm1)을 통해 원래 값 스케일로 복원
        y_pred = np.expm1(y_pred_log)

        y_pred = max(0.0, float(y_pred))
        y_pred = int(round(y_pred))

        preds.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": y_pred,
        })
    df_pred = pd.DataFrame(preds)
    return df_pred


# --- 메인 실행 로직 ---

# --- 데이터 로드 ---
train = pd.read_csv('./train.csv')

# --- 1. S1(탐색)용 pivot_raw 생성 (변경 없음) ---
monthly_raw = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)["value"]
    .sum()
)
monthly_raw["ym"] = pd.to_datetime(
    monthly_raw["year"].astype(str) + "-" + monthly_raw["month"].astype(str).str.zfill(2)
)
pivot_raw = ( # S1용
    monthly_raw
    .pivot(index="item_id", columns="ym", values="value")
    .fillna(0.0)
)
months_dt = pivot_raw.columns.to_list()

# --- S1 실행 (변경 없음) ---
print("--- '원본 값' 기반 공행성 쌍 탐색 시작 ---")
pairs = find_comovement_pairs(pivot_raw, corr_threshold=0.3)
print("탐색된 공행성쌍 수:", len(pairs))


# --- 2. S2(학습)용 pivot_log 생성 (변경 없음) ---
monthly_log = monthly_raw.copy()
monthly_log["value"] = np.log1p(monthly_log["value"])

pivot_log = ( # S2용
    monthly_log
    .pivot(index="item_id", columns="ym", values="value")
    .fillna(0.0)
)
print("S2 학습용 Log 변환 pivot 생성 완료.")


# --- 3. S2 학습 데이터 구축 (변경 없음) ---
df_train_model = build_training_data(pivot_log, pairs, months_dt)
print('생성된 학습 데이터의 shape :', df_train_model.shape)


# --- (3. 회귀모델 학습 - [수정됨] feature_cols 변경) ---

# [수정 4] 'month' 대신 'month_sin', 'month_cos' 사용
feature_cols = [
    'b_t', 'b_t_1', 'a_t_lag', 'max_corr', 'best_lag',
    'b_t_12', 'b_roll_mean_3',
    'month_sin', 'month_cos'
]

if df_train_model.empty:
    print("오류: 학습 데이터가 없습니다. (lag가 너무 크거나, t 시작점이 높을 수 있음)")
    submission = pd.DataFrame(columns=['leading_item_id', 'following_item_id', 'value'])
else:
    train_X = df_train_model[feature_cols].values
    train_y = df_train_model["target"].values

    # [변경 없음] LGBM 파라미터 고정
    reg = LGBMRegressor(random_state=42, n_estimators=500, learning_rate= 0.01,max_depth=5, num_leaves=25, min_child_samples=20, n_jobs=-1)
    print("LGBMRegressor 모델(Sin/Cos 피처) 학습 시작...")
    reg.fit(train_X, train_y)
    print("모델 학습 완료.")


# --- (4. 예측 - [변경 없음]) ---
if df_train_model.empty:
    print("학습된 모델이 없어 예측을 건너뜁니다.")
else:
    print("--- 'Sin/Cos 피처' 모델 예측 시작 ---")
    submission = predict(pivot_log, pairs, reg, months_dt)
    print(submission.head())

# --- 결과 저장 ---
submission.to_csv('LGBM_month.csv', index=False)
print(f"LGBM(S1/S2분리, Log, SinCos) 모델 예측 완료. 저장됨.")

--- '원본 값' 기반 공행성 쌍 탐색 시작 ---


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated si

탐색된 공행성쌍 수: 1833
S2 학습용 Log 변환 pivot 생성 완료.
생성된 학습 데이터의 shape : (54990, 10)
LGBMRegressor 모델(Sin/Cos 피처) 학습 시작...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001817 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1559
[LightGBM] [Info] Number of data points in the train set: 54990, number of used features: 9
[LightGBM] [Info] Start training from score 12.395534
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

0it [00:00, ?it/s]/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packag

  leading_item_id following_item_id    value
0        AANGBULD          APQGTRMF    21956
1        AANGBULD          BEZYMBBT  3130492
2        AANGBULD          DDEXPPXU     4990
3        AANGBULD          DEWLVASR   390409
4        AANGBULD          DNMPSKTB  5002889
LGBM(S1/S2분리, Log, SinCos) 모델 예측 완료. 저장됨.
